# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

1. How many reads map to HA, and how many reads map to NA?

In [119]:
import re
import Bio.SeqIO

#convert the sequecning reads into strings 
seqreads_str = []
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list

In [92]:
def reverse_complement(seq, unk_partner = 'N'):
    """Get reverse complement of a DNA sequence.
    
    Parameters
    -----------
    seq : str
`        Uppercase DNA sequence.
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    rseq = ''
    for a in seq:
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq
        else:
            rseq = unk_partner + rseq
    return rseq

In [164]:
#placing the reverse compliment into a list 
rev = [ ]

for seq in seqreads_str:
    reverse_complement(seq)
    rev.append(reverse_complement(seq))


' '.join(rev[:3])

'TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTTTAGTGCTGCGTTTACAAAACTGGTTTCAAAATTATTAATTTCAGCTTTTTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTTGGCCTGATGGAGCAGAACTCCCTTTCACGATAGATAAATAATAGTGCACCATAGGCGGCCGCACTAAATAACTTAAGC TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTTGTATACTAAGCTGTTGATGGTGTCTTTATGATACCTATTTAGGTTTTTTTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTATGTGTTCCAACGGCTCCTTGCAATGCCGGATTTGCATATAATGATGCACCATAGGCGGCCGCAACCGTGACCAGGAAG TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAAGTTCGATCATTGGCTTCAATACCTAGTGGGAGTTAGATTTTGTATGTTAGTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTTGGCCTGATGGAGCAGAACTCCCTTTCACGATAGATAAATAATAGTGCACCATAGGCGGCCGCGGGGTAATAGCCGTG'

In [127]:
def read_viral_genes(seqs, upstream = 'AGGCGGCCGC', bclen = 16):

    #compile the search pattern
    viral_gene_pattern = re.compile('(?P<gene>[ATCG]{27})' + upstream + "(?P<barcode>[ATCGN]{" + str(bclen) + "})")

    #viral reads
    HA = 'CCGGATTTGCATATAATGATGCACCAT'
    NA = 'CACGATAGATAAATAATAGTGCACCAT'

    #empty dictionary to count number of viral genes 
    gene_count = {'HA_count': 0, 'NA_count': 0, 'neither': 0}

    #loop to search and count the number of HA and NA
    for seq in seqs:
        match = viral_gene_pattern.search(seq)

        if match:
            gene = match.group("gene")
            if gene == HA:
                gene_count['HA_count'] += 1 #counts number of HA
            elif gene == NA:
                gene_count['NA_count'] += 1 #counts number of NA
            else:
                gene_count['neither'] += 1 #counts neither 
    return gene_count 

In [128]:
read_viral_genes(rev)

{'HA_count': 5250, 'NA_count': 3912, 'neither': 248}

2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._

In [161]:
#Define a function that "Identify barcode with known upstream sequence"
def read_barcode(seqs, bclen=16, downstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
    """
    # compiling the reguar expression to run within the function read_barcode
    # We want sequence that starts with a a string of any ATCGs that is 27bp long, followed by the downstream constant sequence, followed by any barcode 
    #that is made of ATCGNs and has a length of 16 right at the end of the sequence
    barcode_re = re.compile("(?P<gene>[ATCG]{27})" + downstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})")
    
    #find the sequences that match the re
    match = barcode_re.search(seq)
    
    #return the sequences that match the patterm as "barcode, gene"
    if match:
        barcode = match.group('barcode')
        gene = match.group('gene')
        return barcode, gene
    else:
        barcode = "not_match"
        gene = "not_match"
        return barcode, gene

In [162]:
#define the HA and NA specific sequences
NA = 'CACGATAGATAAATAATAGTGCACCAT'
HA = 'CCGGATTTGCATATAATGATGCACCAT'

#Set up empty barcode_count dictionaries
barcode_counts_HA = {}
barcode_counts_NA = {}

#make a for loop to read each sequece in the list and 
#1. decide if it matches the barcode regex and return the matches as "barcode, prot" 
#2 assign the sequence as belonging to HA or NA 
#3 count them up in a dictionary
for seq in rev:
    barcode, gene = read_barcode(seq, bclen = 16)

    if gene == NA:
        if barcode in barcode_counts_NA:
            barcode_counts_NA[barcode] += 1
        else:
            barcode_counts_NA[barcode] = 1
    elif gene == HA:
        if barcode in barcode_counts_HA:
            barcode_counts_HA[barcode] += 1
        else:
            barcode_counts_HA[barcode] = 1

In [163]:
#find the barcode with the most counts for each prot (NA or HA) and print that barcode and the count 
max_NA_barcode = max(barcode_counts_NA)
max_NA_count = max(barcode_counts_NA.values())

max_HA_barcode = max(barcode_counts_HA)
max_HA_count = max(barcode_counts_HA.values())


print("the NA barcode with the most counts is", max_NA_barcode, "with", max_NA_count, "counts")
print("the HA barcode with the most counts is", max_HA_barcode, "with", max_HA_count, "counts")

the NA barcode with the most counts is TTTCTCCCGCATCTCG with 152 counts
the HA barcode with the most counts is TTTGTCCCTTGAACCC with 155 counts
